<a href="https://colab.research.google.com/github/arthurantonio/PySparkUdemy/blob/main/Desafio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#!pip install pyspark==3.3.2

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [34]:
import pyspark.sql.functions as F
from pyspark.sql import SparkSession, Window

In [6]:
spark = (
  SparkSession.builder.appName('Desafio')
  .config('spark.sql.repl.eagerEval.enabled', True)
  .getOrCreate()
)

##Titanic

In [63]:
df_titanic  = spark.read.csv('/content/drive/MyDrive/PySpark/data/Desafio/titanic.csv', header=True).alias('titanic')

In [64]:
df_titanic

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,0,3,"Braund, Mr. Owen ...",male,22,1,0,A/5 21171,7.25,null,S
2,1,1,"Cumings, Mrs. Joh...",female,38,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. ...",female,26,0,0,STON/O2. 3101282,7.925,null,S
4,1,1,"Futrelle, Mrs. Ja...",female,35,1,0,113803,53.1,C123,S
5,0,3,"Allen, Mr. Willia...",male,35,0,0,373450,8.05,null,S
6,0,3,"Moran, Mr. James",male,null,0,0,330877,8.4583,null,Q
7,0,1,"McCarthy, Mr. Tim...",male,54,0,0,17463,51.8625,E46,S
8,0,3,"Palsson, Master. ...",male,2,3,1,349909,21.075,null,S
9,1,3,"Johnson, Mrs. Osc...",female,27,0,2,347742,11.1333,null,S
10,1,2,"Nasser, Mrs. Nich...",female,14,1,0,237736,30.0708,null,C


In [65]:
(
    df_titanic
    .groupBy('Survived')
    .agg(
        F.count('Survived').alias('Total')
    )
    .withColumn('Porcentagem', F.round(F.col('Total')/df.count()*100, 2))
)

Survived,Total,Porcentagem
0,549,61.62
1,342,38.38


In [66]:
(
    df_titanic
    .groupBy('Survived')
    .agg(
        F.round(F.mean('Age'), 2).alias('Media de Idade')
    )
)

Survived,Media de Idade
0,30.63
1,28.34


In [67]:
(
    df_titanic
    .groupBy(['Pclass','Survived'])
    .agg(
        F.count('Survived').alias('Total')
    )
)

Pclass,Survived,Total
1,1,136
2,0,97
3,1,119
3,0,372
2,1,87
1,0,80


In [68]:
window_spec = Window.partitionBy('Sex')

(
    df_titanic
    .groupBy(['Sex','Survived'])
    .agg(
        F.count('Survived').alias('Total')
    )
    .withColumn('TotalPorSexo', F.sum('Total').over(window_spec))
    .withColumn('Porcentagem', F.round(100 * (F.col('Total') / F.col('TotalPorSexo')), 2))
    .filter(F.col('Sex') == 'female')
)

Sex,Survived,Total,TotalPorSexo,Porcentagem
female,0,81,314,25.8
female,1,233,314,74.2


In [69]:
(
    df_titanic
    .groupBy(['Survived'])
    .agg(
        F.count('Survived').alias('Total'),
        F.sum('Fare').alias('Valor acumulado das passagens')
    )
    .withColumn('Valor médio da passagem', F.round(F.col('Valor acumulado das passagens')/F.col('Total'), 2))
)

Survived,Total,Valor acumulado das passagens,Valor médio da passagem
0,549,12142.719899999987,22.12
1,342,16551.229399999997,48.4


In [70]:
df_titanic.describe()

summary,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891,891,891,891,891,714,891,891,891,891,204,889
mean,446.0,0.3838383838383838,2.308641975308642,null,null,29.69911764705882,0.5230078563411896,0.38159371492704824,260318.54916792738,32.2042079685746,null,null
stddev,257.3538420152301,0.48659245426485753,0.8360712409770491,null,null,14.526497332334035,1.1027434322934315,0.8060572211299488,471609.26868834975,49.69342859718089,null,null
min,1,0,1,"""Andersson, Mr. A...",female,0.42,0,0,110152,0,A10,C
max,99,1,3,"van Melkebeke, Mr...",male,9,8,6,WE/P 5735,93.5,T,S


In [71]:
(
    df_titanic
    .groupBy(['Sex'])
    .agg(
        F.count('Sex').alias('Total')
    )
    .withColumn('Porcentagem', F.round(F.col('Total')/df.count()*100, 2))
)

Sex,Total,Porcentagem
female,314,35.24
male,577,64.76


In [72]:
window_spec = Window.partitionBy('Pclass')

(
    df_titanic
    .groupBy(['Pclass','Survived'])
    .agg(
        F.count('Survived').alias('Total')
    )
    .withColumn('TotalPorClasse', F.sum('Total').over(window_spec))
    .withColumn('Porcentagem', F.round(100 * (F.col('Total') / F.col('TotalPorClasse')), 2))
)

Pclass,Survived,Total,TotalPorClasse,Porcentagem
1,1,136,216,62.96
1,0,80,216,37.04
2,0,97,184,52.72
2,1,87,184,47.28
3,1,119,491,24.24
3,0,372,491,75.76


##Shopping

In [51]:
df_shopping = spark.read.csv('/content/drive/MyDrive/PySpark/data/Desafio/Mall_Customers.csv', header=True).alias('shopping')

In [52]:
df_shopping

CustomerID,Gender,Age,Annual Income (k$),Spending Score (1-100)
1,Male,19,15,39
2,Male,21,15,81
3,Female,20,16,6
4,Female,23,16,77
5,Female,31,17,40
6,Female,22,17,76
7,Female,35,18,6
8,Female,23,18,94
9,Male,64,19,3
10,Female,30,19,72


In [53]:
df_shopping.describe()

summary,CustomerID,Gender,Age,Annual Income (k$),Spending Score (1-100)
count,200,200,200,200,200
mean,100.5,null,38.85,60.56,50.2
stddev,57.879184513951124,null,13.96900733155888,26.26472116527124,25.823521668370173
min,1,Female,18,101,1
max,99,Male,70,99,99


In [54]:
(
    df_shopping
    .groupBy(['Gender'])
    .agg(
        F.count('Gender').alias('Total')
    )
)

Gender,Total
Female,112
Male,88


In [57]:
(
    df_shopping
    .groupBy(['Annual Income (k$)'])
    .agg(
        F.count('Annual Income (k$)').alias('Total')
    )
    .filter(F.col('Annual Income (k$)') >= 80)
).agg(F.sum('Total').alias('SomaTotal')).collect()[0]['SomaTotal']

38

In [62]:
(
    df_shopping
    .withColumn('classe',
          F.when(F.col('Annual Income (k$)') < 80, 'Menor que 80')
          .otherwise('Maior ou igual a 80')
    )
    .groupBy(['classe'])
    .agg(
        F.count('classe').alias('Total')
    )
)

classe,Total
Menor que 80,162
Maior ou igual a 80,38


In [60]:
(
    df_shopping
    .groupBy('Spending Score (1-100)')
    .agg(
        F.count('Spending Score (1-100)').alias('Total')
    )
    .orderBy(F.col('Total').desc())
    .limit(1)
)

Spending Score (1-100),Total
42,8
